In [2]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model
import random
import glob
import tensorflow as tf
import os as os 
os.chdir('/home/himanshu/cats_dogs/')

In [ ]:
# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = '/home/himanshu/cats_dogs/train'
validation_data_dir = '/home/himanshu/cats_dogs/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 20
batch_size = 16

# build the VGG16 network
base_model = applications.VGG16(weights='imagenet', include_top=False,input_shape=(150, 150, 3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
model = Model(input= base_model.input, output= top_model(base_model.output))
print('Top model added with pre-trained fc layer weights.')
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

# fine-tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples//batch_size)

Model loaded.


/home/himanshu/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("se..., inputs=Tensor("in...)`


Top model added with pre-trained fc layer weights.
Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Epoch 1/20
125/125 [==============================] - 293s 2s/step - loss: 0.2669 - acc: 0.9390 - val_loss: 0.5951 - val_acc: 0.9050
Epoch 2/20
125/125 [==============================] - 294s 2s/step - loss: 0.2441 - acc: 0.9385 - val_loss: 0.5951 - val_acc: 0.9050
Epoch 3/20
125/125 [==============================] - 287s 2s/step - loss: 0.3323 - acc: 0.9230 - val_loss: 0.5951 - val_acc: 0.9050
Epoch 4/20
125/125 [==============================] - 289s 2s/step - loss: 0.2392 - acc: 0.9470 - val_loss: 0.5951 - val_acc: 0.9050
Epoch 5/20
125/125 [==============================] - 290s 2s/step - loss: 0.3448 - acc: 0.9295 - val_loss: 0.5951 - val_acc: 0.9050
Epoch 6/20
 63/125 [==============>...............] - ETA: 1:46 - loss: 0.3198 - acc: 0.9187